In [ ]:
%pip install python-dotenv langchain langchain-openai langchain-pinecone pinecone-client langchain-community

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [5]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
pc.create_index(
    name="langchain-demo",
    dimension=1536,  # Dimension of the OpenAI embeddings
    metric="cosine",  # Similarity metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "langchain-demo",
    "metric": "cosine",
    "host": "langchain-demo-szyj9c0.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [6]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)

In [7]:
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore

doc1 = Document(page_content="Building an exciting new project with LangChain - come check it out!",
                metadata={"source": "tweet"})
doc2 = Document(page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
                metadata={"source": "news"})
doc3 = Document(page_content="LangGraph is the best framework for building stateful, agentic applications!",
                metadata={"source": "tweet"})

vector_store = PineconeVectorStore(
    index_name="langchain-demo",
    embedding=embedding
)

vector_store.add_documents([doc1, doc2, doc3])



['5aab0776-0b59-477a-a57c-ca6f30997507',
 '26d7fae2-7740-43c7-88e1-1d5de17b1810',
 'e704ec5e-f1d0-4aed-9b9d-76de63080ff1']

In [8]:
pc.Index("langchain-demo").describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3,
 'vector_type': 'dense'}

In [27]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
pc.create_index(
    name="movie-index",
    dimension=1536,  # Dimension of the OpenAI embeddings
    metric="cosine",  # Similarity metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "movie-index",
    "metric": "cosine",
    "host": "movie-index-szyj9c0.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [28]:
movie_index = pc.Index("movie-index")
movie_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
%pip install pandas

In [29]:
import pandas as pd

data = [
    {
        "title": "응답하라 1988",
        "year": 2015,
        "genre": ["드라마", "코미디"],
        "director": "신원호",
        "actors": ["혜리", "박보검", "류준열"],
        "rating": 9.2,
        "synopsis": "1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
    },
    {
        "title": "기생충",
        "year": 2019,
        "genre": ["드라마", "스릴러"],
        "director": "봉준호",
        "actors": ["송강호", "이선균", "조여정"],
        "rating": 8.6,
        "synopsis": "가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화."
    }
]
df = pd.DataFrame(data)
df.head(2)

,title,year,genre,director,actors,rating,synopsis
0,응답하라 1988,2015,"[드라마, 코미디]",신원호,"[혜리, 박보검, 류준열]",9.2,"1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
1,기생충,2019,"[드라마, 스릴러]",봉준호,"[송강호, 이선균, 조여정]",8.6,가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.


In [30]:
texts = df['synopsis'].tolist()
texts

['1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.',
 '가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.']

In [31]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)

In [32]:
texts = []
metas = []
ids = []

for i, sample in enumerate(data):
    text = sample['synopsis']
    metadata = {
        "title": sample['title'],
        "year": sample['year'],
        "genre": sample['genre'],
        "director": sample['director'],
        "actors": sample['actors'],
        "rating": sample['rating'],
        "text": sample['synopsis']
    }

    ids.append(f"movie-{i}")
    texts.append(text)
    metas.append(metadata)

print(ids)
print(texts)
print(metas)

vectors = embeddings.embed_documents(texts)
print(len(vectors[0]))

['movie-0', 'movie-1']
['1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.', '가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.']
[{'title': '응답하라 1988', 'year': 2015, 'genre': ['드라마', '코미디'], 'director': '신원호', 'actors': ['혜리', '박보검', '류준열'], 'rating': 9.2, 'text': '1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.'}, {'title': '기생충', 'year': 2019, 'genre': ['드라마', '스릴러'], 'director': '봉준호', 'actors': ['송강호', '이선균', '조여정'], 'rating': 8.6, 'text': '가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.'}]
1536


In [33]:
movie_index.upsert(
    vectors=zip(ids, vectors, metas)
)

{'upserted_count': 2}

In [45]:
vector_store = PineconeVectorStore(
    index_name="movie-index",
    embedding=embeddings
)

query = "드라마 추천해줘"

results = vector_store.similarity_search(
    query, 
    k=2
)
for result in results:
    print(result)

page_content='1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.' metadata={'actors': ['혜리', '박보검', '류준열'], 'director': '신원호', 'genre': ['드라마', '코미디'], 'rating': 9.2, 'title': '응답하라 1988', 'year': 2015.0}
page_content='가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.' metadata={'actors': ['송강호', '이선균', '조여정'], 'director': '봉준호', 'genre': ['드라마', '스릴러'], 'rating': 8.6, 'title': '기생충', 'year': 2019.0}


In [46]:
movies = [
    {
        "id": "movie1",
        "title": "7번방의 선물",
        "year": 2013,
        "genre": "드라마",
        "description": "억울한 누명을 쓰고 교도소에 수감된 아빠와 그의 어린 딸의 감동적인 스토리"
    },
    {
        "id": "movie2",
        "title": "미나리",
        "year": 2020,
        "genre": "드라마",
        "description": "한국계 미국인 가족의 따뜻하고 감성적인 성장 이야기"
    },
    {
        "id": "movie3",
        "title": "기생충",
        "year": 2019,
        "genre": "드라마",
        "description": "가난한 가족과 부자 가족 사이의 빈부격차를 그린 사회 풍자 드라마"
    },
    {
        "id": "movie4",
        "title": "범죄도시",
        "year": 2017,
        "genre": "범죄",
        "description": "형사가 범죄 조직을 소탕하는 범죄 액션 영화"
    },
    {
        "id": "movie5",
        "title": "범죄도시 2",
        "year": 2022,
        "genre": "범죄",
        "description": "형사와 범죄 조직의 대결을 그린 범죄 액션 영화의 속편"
    },
    {
        "id": "movie6",
        "title": "헤어질 결심",
        "year": 2022,
        "genre": "범죄",
        "description": "산에서 발생한 의문의 죽음(살인 사건)을 수사하던 형사가 피의자에게 이끌리며 벌어지는 미스터리 멜로 영화"
    },
    {
        "id": "movie7",
        "title": "다만 악에서 구하소서",
        "year": 2020,
        "genre": "범죄",
        "description": "청부 살인업자와 범죄 조직의 마지막 거래를 그린 범죄 액션 영화"
    }
]

In [47]:
descriptions = [movie['description'] for movie in movies]
movie_vectors = embeddings.embed_documents(descriptions)

In [48]:
pc.create_index(
    name="movie-index2",
    dimension=1536, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "movie-index2",
    "metric": "cosine",
    "host": "movie-index2-szyj9c0.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [49]:
movie_index2 = pc.Index("movie-index2")

In [50]:
vector_data = []

for movie, vector in zip(movies, movie_vectors):
    metadata = {
        "title": movie['title'],
        "year": movie['year'],
        "genre": movie['genre'],
        "description": movie['description']
    }
    vector_data.append((movie['id'], vector, metadata))

In [56]:
query_text = "2020년 이후 개봉된 범죄와 관련된 영화를 추천해줘."

query_vector = embeddings.embed_query(query_text)
results = movie_index2.query(
    vector=query_vector,
    top_k=3,
    include_metadata=True,
    filter={
        "year": {"$gte": 2020}
    }
)

for result in results['matches']:
    print(f"ID: {result['id']}, Score: {result['score']}, Metadata: {result['metadata']}")

ID: movie5, Score: 0.456922, Metadata: {'description': '형사와 범죄 조직의 대결을 그린 범죄 액션 영화의 속편', 'genre': '범죄', 'title': '범죄도시 2', 'year': 2022.0}
ID: movie7, Score: 0.446524888, Metadata: {'description': '청부 살인업자와 범죄 조직의 마지막 거래를 그린 범죄 액션 영화', 'genre': '범죄', 'title': '다만 악에서 구하소서', 'year': 2020.0}
ID: movie6, Score: 0.265735477, Metadata: {'description': '산에서 발생한 의문의 죽음(살인 사건)을 수사하던 형사가 피의자에게 이끌리며 벌어지는 미스터리 멜로 영화', 'genre': '범죄', 'title': '헤어질 결심', 'year': 2022.0}
